## Dependencies

In [18]:
import pandas as pd
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pickle

## Loading the data

In [19]:
train_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Customer Feedback Classification/absa/src/dataset/train - train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Customer Feedback Classification/absa/src/dataset/test - test.csv')
test_data

,text,aspect
0,improve your customer service and product avai...,Customer service
1,"functionality is great, almost as in desktop v...",mobile version
2,but it keeps starting from zoomed in and then ...,zoomed
3,hey marilyn thanks for your answer the soc2 ty...,Security
4,@delanovc @zoom @airtable @notionhq @calendly ...,apple
...,...,...
995,in a database where i have multiple views enab...,views
996,the mere act of opening the @notionhq tab make...,opening
997,i have shared it to web now and added the link...,web
998,why the members who regularly buy milk has to ...,service charge.


## Processing

In [20]:
print('-----Missing Values-----')
print(train_data.isnull().sum(), '\n')
print('---------Classes---------')
train_data['label'].value_counts()

-----Missing Values-----
text      0
aspect    0
label     0
dtype: int64 

---------Classes---------


0    1680
1    1294
2    1026
Name: label, dtype: int64

### One-Hot Encoding Texts

In [21]:
enc_train_data = pd.get_dummies(train_data, columns=['text', 'aspect'])
enc_train_data

,label,"text_""My Task"" tab doesn't save the custom filter after I restart the app.","text_""allow support access""",text_# anything else you can share to help us verify these details?,"text_# anything else you can share to help us verify these details? # if you have attachments to share, include them here!","text_# notion is an equal opportunity organization and will not allow discrimination based upon age, ethnicity, ancestry, gender, national origin, disability, race, size, religion, sexual orientation, socioeconomic background, or any other status prohibited by applicable law.",text_# what is your workspace domain?,"text_#inapp_contactus name: david tabellion email: company: dayone company size: 1 - 100 type: other submission details: hi notion team, we absolutely love your product and use it within in our company on a daily basis.",text_$$\nyour math\n$$,text_&grocerry varities,text_(i do not have apple stuff.),text_(i use columns all the time),"text_(page permissions, workspace name, page name, keywords, urls, alternate emails, web sessions, login method) - page name: journal (it was the default journal page that was available) [image ""image.png?expires=&amp;signature=da0a4344382a89a935ec02b5e38355c186b959be379c78d818ea1eb17b765fd3""] *if the user deleted their workspace and/or account* - user ids (current and deleted, if any): n/a - workspace ids (if user deleted workspace): n/a - web sessions on current account: 4","text_(page permissions, workspace name, page name, keywords, urls, alternate emails, web sessions, login method) - page url: - web sessions on this account: 1 - support access granted.",text_(she/her) design &amp; operations manager,text_(whenever we launch it) license to resell videos video resolution in 1080p full access to all editing,text_(x86_64) web clipper plugin version,"text_* Update changed the way tables/views scroll sideways, leaving a descriptive header for each row, but I'll be OK with it eventually, since I see what the goal is.","text_*meaning, if i later upgrade to the paid plan.",text_+ aesthetic database text + tiled header (more r.,text_+ in certain tables is gone after update,"text_+64 | m: | | linkedin profile from: community &lt;&gt; sent: thursday, 29 april 2021","text_, bc this account needs a pricing-model.","text_, i did not realize my watermark logo was on - is there a way to remove it all at once?","text_, i had used a coupon code but none of the history in the app could figure out where it got vanished and none of your agents could track it, and you guys wanted me to keep a snapshot of that coupon code when i applied it.","text_, like alowing copy and paste for the font search box..","text_, notion team notion.so, the all-in-one workspace for your notes, tasks, wikis, and databases.","text_, vga_640x480 (30fps) video format: mov image resolution: vga,24m,20m,14m,12m,8m,5m,3m,2m","text_, you gota know all the tools it offer to take the best out of it, but if you just what to take practical and visual notes this is the perfect app for you.",text_- Dark Mode\n- Apple Watch App\n- Widgets\n,text_- Displaying issues on ipad in landscape mode\n- no ability to create teams on iOS \n- not possible to change the default view mode of a project...\n,text_- UI for Ipad pro is a joke.,"text_- can you send me an email or link, so that you are attributed to it?","text_- old review best productivity system but android app sucks, pc version is best.","text_- once you get back to me on mail, ill help you make the payment and then put you on a lifetime plan directly","text_-&gt; i want to learn for the future whether we as a team did something wrong (whether a teammate accidently deleted the file -&gt; therefore it would be good to get the mailadress of ""kaiser""), or we got hacked, or whether there was a failure in the notion system.",text_-1 star because does not support hand writing or sketching with the apple pencil on,"text_-calendar view for databases is also ultra limited, you can not s

#### Splitting

In [22]:
ohe_X_train = enc_train_data.drop('label', axis=1)
y_train = enc_train_data.label

In [23]:
pickle.dump(ohe_X_train, open('/content/drive/MyDrive/Colab Notebooks/Customer Feedback Classification/absa/src/pickled files/OHE_X_train.pkl','wb'))

### Word Embeddings

#### Train

In [24]:
we_X_train = train_data['text']
y_train = train_data.label

In [25]:
tk = Tokenizer(len(y_train))
tk.fit_on_texts(we_X_train)
we_X_train = tk.texts_to_sequences(we_X_train)
we_X_train = pad_sequences(we_X_train, maxlen=28, truncating='post', padding='post')
we_X_train[0]

array([ 20,  13, 274, 455, 128, 282, 611,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0], dtype=int32)

#### Test

In [26]:
X_test = test_data['text']
tk = Tokenizer(len(test_data))

tk.fit_on_texts(X_test)
X_test = tk.texts_to_sequences(X_test)
X_test = pad_sequences(X_test, maxlen=28, truncating='post', padding='post')
X_test[0]

array([262,  27,  85,  51,   4, 141, 473,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0], dtype=int32)

In [27]:
pickle.dump(we_X_train, open('/content/drive/MyDrive/Colab Notebooks/Customer Feedback Classification/absa/src/pickled files/WE_X_train.pkl', 'wb'))
pickle.dump(y_train, open('/content/drive/MyDrive/Colab Notebooks/Customer Feedback Classification/absa/src/pickled files/y_train.pkl', 'wb'))
pickle.dump(X_test, open('/content/drive/MyDrive/Colab Notebooks/Customer Feedback Classification/absa/src/pickled files/X_test.pkl', 'wb'))